# Tutorial for srfpython

Overview  
 + I/ create a 1-D depth model  
 + II/ compute dispersion curves   
     + II.1/ use in a python program  
     + II.2/ use in command line   
 - III/ depth inversion  
 - III.1/ Program HerrMet  
 - III.2/ Application  
     + III.2.1/ Target data  
     + III.2.2/ Parameterization  
     + III.2.3/ Run inversion  
     + III.2.4/ Extract results

In [ ]:
from srfpython import *

## I/ create a 1-D depth model

In [ ]:
# create 1-D depth model
ztop = [0.00, 0.25, 0.45, 0.65, 0.85, 1.05, 1.53, 1.80] #km, top layer depth
vp   = [1.85, 2.36, 2.63, 3.15, 3.71, 4.54, 5.48, 5.80] #km/s
vs   = [0.86, 1.10, 1.24, 1.47, 1.73, 2.13, 3.13, 3.31] #km/s
rh   = [2.47, 2.47, 2.47, 2.47, 2.47, 2.58, 2.58, 2.63] #g/cm3

dm = depthmodel_from_arrays(ztop, vp, vs, rh)
print dm #__str__ returns the file content at mod96 format, see Herrmann doc (CPS), see also dm.write96

In [ ]:
#write it as a file at mod96 format (CPS)
dm.write96('dmtuto.mod96')

In [ ]:
# display
plt.figure(figsize=(2, 4))
dm.show(gca())
gca().set_title('figure 1 : dmtuto.mod96')
gca().grid(True)
plt.legend();

## II/ compute dispersion curves 

### II.1/ use in a python program

In [ ]:
print help(dispersion_2)

In [ ]:
# define the dipsersion curves to compute
#          Wave(R/L) Type(C/U) Mode    Frequency array (Hz)             
Curves = [('R',      'U',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('R',      'U',      1,      freqspace(0.2, 3.5, 35, "log")), 
          ('R',      'C',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('R',      'C',      1,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'U',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'U',      1,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'C',      0,      freqspace(0.2, 3.5, 35, "log")), 
          ('L',      'C',      1,      freqspace(0.2, 3.5, 35, "log"))] 

# compute dispersion curves
with Timer('dispersion'):
    out = list(dispersion_2(ztop, vp, vs, rh, Curves))

# display results
ax = plt.gca()
for w, t, m, fs, us in out:
    ax.loglog(1. / fs, us, '+-', label = "%s%s%d" % (w, t, m))
ax.set_xlabel('period (s)')
ax.set_ylabel('velocity (km/s)')    
ax.grid(True, which = "major")
ax.grid(True, which = "minor")
logtick(ax, "xy")
ax.set_title('figure 2 : Herrmann.py demo')

plt.legend()
plt.show()

### II.2/ use in command line 

In [ ]:
# compute dispersion curves, and save as surf96 file
import os
os.system('rm -f dmtuto*.surf96')
%run -i ../bin/m96 --disp dmtuto.mod96 \
    -LC0 .1 10 30 plog \
    -RC1 .1 10 30 plog \
    -RU0 .1 10 30 plog \
    -save dmtuto.surf96


In [ ]:
# display output
%run -i ../bin/s96 --show dmtuto.surf96 -inline

see also programs s96 and m96 that provide more manipulation tools  
for depth models and surface wave dispersion curves

In [ ]:
%run -i ../bin/m96 --help

In [ ]:
%run -i ../bin/s96 --help

## III/ depth inversion

### III.1/ Program HerrMet

In [ ]:
%run -i ../bin/HerrMet --help

In [ ]:
%run -i ../bin/HerrMet -ex

### III.2/ Application

we propose to invert the synthetic data generated in section I (dmtuto.surf96) and compare the inversion result to the actual model used to synthetize the data (i.e. dmtuto.mod96)

In [ ]:
assert os.path.exists("./dmtuto.surf96")

#### III.2.1/ Target data

In [ ]:
# get the target dispersion curves, resample it between 0.25-1 Hz 
# with 7 samples spaced logarithmically in period domain
# adjust uncertainties to 0.2*velocity (i.e. constant uncertainty in logaritmic domain), 
# overwrite target if exists (_HerrMet.target) 
%run -i ../bin/HerrMet --target dmtuto.surf96 \
            -resamp 0.25 1.0 7 plog \
            -lunc 0.2 \
            -ot
%run -i ../bin/s96 --show _HerrMet.target -inline
#or ./bin/HerrMet --target dmtuto.surf96

#### III.2.2/ Parameterization

In [ ]:
# build the parameter file from an existing depthmodel (dmtuto.mod96),
# use 4 layers down to 3 km, 
# use parametrization mode mZVSPRRH (means top depth, VS, VP/VS and density in each layer)
# require vp, vs and density to be growing
# overwrite paramfile if exists (_HerrMet.param) and display
%run -i ../bin/HerrMet --param 4 3. \
            -basedon dmtuto.mod96 -m96 dmtuto.mod96 \
            -t  mZVSPRRH \
            -growing \
            -op --disp -inline

> red dashed curves = prior boundaries  
> green duspersion curves = target data  
> purple model = actual model used to generate the synthetic data  

> Note that at this step, the boundaries for each parameter (red dashed curves)   
> are the same (because VINF=VSUP in _HerrMet.param) : i.e. all parameters are locked  
> one need to adjust the VINF, VSUP boundaries for all parameters to invert  
>
> you may do it manually (edit _HerrMet.param),   
> here I do it with programming tools for tutorial

In [ ]:
#load the parameter file, find lines related to top depth and to VS
from tetedenoeud import AsciiFile
A = AsciiFile('_HerrMet.param')

IZ  = np.asarray(["Z"  in _ for _ in A['KEY']], bool) #lines corresponding to Z parameters
IVS = np.asarray(["VS" in _ for _ in A['KEY']], bool) #lines corresponding to VS parameters
IPR = np.asarray(["PR" in _ for _ in A['KEY']], bool) #lines corresponding to VP/VS parameters

In [ ]:
#change parameter boundaries (decrease VINF and increase VSUP), overwrite _HerrMet.param
A['VINF'][IVS] = [0.55, 0.78, 1.53, 1.65]
A['VSUP'][IVS] = [2.22, 3.15, 4.00, 4.00]
A['VINF'][IZ]  = [-.31, -1.5, -3.1]
A['VSUP'][IZ]  = [-.11, -.79, -2.9]
print A
A.write('_HerrMet.param')

In [ ]:
#display the parameterization
#note that the boundaries now allow VS and Zop to vary between the red dashed lines
%run -i ../bin/HerrMet --disp -inline -m96 ./dmtuto.mod96

#### III.2.3/ Run inversion

In [ ]:
#run inversion with 4 markov chains, keep 1000 models per chain
%run -i ../bin/HerrMet -w 4 -taskset "0-3" \
        --run restart -nchain 4 -nkeep 1000 -verbose off

In [ ]:
#display the best 1000 models found and their image in the dataspace 
#(recomputed with thinner resolution : overdisp)
#compute the median and std of the resulting population at each depth (blue)
#display
%run -i ../bin/HerrMet \
        --disp 1000 1 \
            -overdisp -verbose off \
            -range \
            -inline \
            -m96 dmtuto.mod96

> red dashed curves = prior boundaries  
> green duspersion curves = target data  
> gray models = best models sorted by increaseing likelyhood  
> gray dispersion curves = corresponding data, recomputed with higher resolution 
> blue lines = median (thick), 16% 84% percentiles (thin) computed from the best models displayed  
> purple model = actual model used to generate the synthetic data  


#### III.2.4/ Extract results

In [ ]:
#compute the median and std of the best 1000 models found, save it as mod96 files
#named _HerrMet.p0.16.mod96,_HerrMet.p0.50.mod96 and _HerrMet.p0.84.mod96
%run -i ../bin/HerrMet --extract 1000 1 

In [ ]:
%run -i ../bin/m96 --show _HerrMet.p*.mod96 dmtuto.mod96 -inline